# Amazon Bedrock Knowledge Bases가 지원하는 쿼리 재구성

RAG 기반 GenAI 애플리케이션을 개발할 때 품질, 비용, 지연 시간을 최적화하는 것이 가장 중요한 요소 중 일부입니다. Foundation Model(FM)에 대한 입력 쿼리는 많은 질문과 복잡한 관계를 포함하는 매우 복잡한 경우가 많습니다. 이러한 복잡한 쿼리의 경우, 임베딩 단계에서 쿼리의 중요한 구성 요소가 가려지거나 희석될 수 있어, 쿼리의 모든 측면에 대한 컨텍스트를 제공하지 못하는 청크가 검색될 수 있습니다. 이로 인해 RAG 애플리케이션에서 원하는 것보다 낮은 품질의 응답이 생성될 수 있습니다.

이제 쿼리 재구성을 통해 복잡한 입력 프롬프트를 여러 개의 하위 쿼리로 분해할 수 있습니다. 이러한 하위 쿼리들은 각각 관련 청크를 검색하는 자체 검색 단계를 거치게 됩니다. 그 결과로 나온 청크들은 FM에 전달되어 응답을 생성하기 전에 함께 풀링되고 순위가 매겨집니다. 쿼리 재구성은 애플리케이션이 실제 운영 환경에서 직면할 수 있는 복잡한 쿼리의 정확도를 높이는 데 도움이 될 수 있는 또 다른 도구입니다.

# 1. Notebook setup
시작하기 위해 호환되는 역할과 컴퓨팅 환경으로 아래 단계들을 따르세요

In [29]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [30]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [31]:
import os
import sys
import time
import boto3
import logging
import pprint
import json

# Set the path to import module
from pathlib import Path
current_path = Path().resolve()
current_path = current_path.parent
if str(current_path) not in sys.path:
    sys.path.append(str(current_path))
# Print sys.path to verify
# print(sys.path)

from utils.knowledge_base import BedrockKnowledgeBase

In [32]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region =  session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

('us-west-2', '211125368524')

In [33]:
import time

# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"
knowledge_base_name_standard = 'standard-kb'
knowledge_base_description = "Octank 10k KB"
bucket_name = f'{knowledge_base_name_standard}-{suffix}'

foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

data_source=[{"type": "S3", "bucket_name": bucket_name}]

## 2. 고정 청킹 전략으로 지식 베이스 생성

In [34]:
knowledge_base_standard = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name_standard}-{suffix}',
    kb_description=knowledge_base_description,
    data_sources=data_source,
    chunking_strategy = "FIXED_SIZE", 
    suffix = f'{suffix}-f'
)

Step 1 - Creating or retrieving S3 bucket(s) for Knowledge Base documents
['standard-kb-9085801']
buckets_to_check:  ['standard-kb-9085801']
Creating bucket standard-kb-9085801
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_9085801-f) and Policies
Step 3a - Creating OSS encryption, network and data access policies
Step 3b - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '320',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Tue, 29 Jul 2025 08:58:04 '
                                                 'GMT',
                                         'x-amzn-requestid': 'e8c4ffec-b295-488a-9df2-43c886cb55eb'},
                        'HTTPStatusCode': 200,
                        'RequestId': 'e8c4ffec-b29

[2025-07-29 08:59:34,930] p18163 {base.py:258} INFO - PUT https://638t7cxc03293rviymx9.us-west-2.aoss.amazonaws.com:443/bedrock-sample-rag-index-9085801-f [status:200 request:0.335s]



Creating index:
{ 'acknowledged': True,
  'index': 'bedrock-sample-rag-index-9085801-f',
  'shards_acknowledged': True}
Step 4 - Will create Lambda Function if chunking strategy selected as CUSTOM
Not creating lambda function as chunking strategy is FIXED_SIZE
Step 5 - Creating Knowledge Base
{ 'createdAt': datetime.datetime(2025, 7, 29, 9, 0, 35, 44864, tzinfo=tzlocal()),
  'description': 'Octank 10k KB',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-west-2:211125368524:knowledge-base/TXYLN3R29K',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v2:0'}},
  'knowledgeBaseId': 'TXYLN3R29K',
  'name': 'standard-kb-9085801',
  'roleArn': 'arn:aws:iam::211125368524:role/AmazonBedrockExecutionRoleForKnowledgeBase_9085801-f',
  'status': 'CREATING',
  'storageConfiguration': { 'opensearchServerlessConfiguration': { 'collectionArn': 

## 2.1 데이터셋을 Amazon S3에 업로드
이제 지식 베이스를 생성했으니, `Octank financial 10K` 보고서 데이터셋으로 채워보겠습니다. Knowledge Base 데이터 소스는 연결된 S3 버킷에서 데이터를 사용할 수 있어야 하며, `StartIngestionJob API` 호출을 사용하여 데이터의 변경 사항을 지식 베이스와 동기화할 수 있습니다. 이 예제에서는 우리의 헬퍼 클래스를 통해 API의 [boto3 추상화](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/start_ingestion_job.html)를 사용할 것입니다.

먼저 `dataset` 폴더에 있는 메뉴 데이터를 s3에 업로드해보겠습니다.

In [35]:
import os

def upload_directory(path, bucket_name):
    for root, dirs, files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root, file)
            if file not in ["LICENSE", "NOTICE", "README.md"]:
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload, bucket_name, file)
            else:
                print(f"Skipping file {file_to_upload}")

upload_directory("../synthetic_dataset", bucket_name)


uploading file ../synthetic_dataset/2024_population.pdf to standard-kb-9085801
uploading file ../synthetic_dataset/.ipynb_checkpoints/2024_population-checkpoint.pdf to standard-kb-9085801


In [36]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base_standard.start_ingestion_job()

job 1 started successfully

{ 'dataSourceId': 'O7XU7FC8NC',
  'ingestionJobId': 'SDRYBJWK6G',
  'knowledgeBaseId': 'TXYLN3R29K',
  'startedAt': datetime.datetime(2025, 7, 29, 9, 1, 8, 566751, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 2,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 2},
  'status': 'COMPLETE',
  'updatedAt': datetime.datetime(2025, 7, 29, 9, 1, 53, 582196, tzinfo=tzlocal())}
........................................

In [37]:
kb_id = knowledge_base_standard.get_knowledge_base_id()

'TXYLN3R29K'


# 3. 쿼리 재구성 실제 적용
이 노트북에서는 쿼리 재구성의 이점을 얻을 수 있는 간단한 쿼리와 더 복잡한 쿼리를 조사하고, 이것이 생성된 응답에 어떤 영향을 미치는지 살펴보겠습니다.

## 복잡한 프롬프트

기능을 시연하기 위해, Octank 10K 재무 문서에 포함된 정보에 대해 여러 가지를 요청하는 쿼리를 살펴보겠습니다. 이 쿼리는 의미론적으로 관련되지 않은 몇 가지 요청을 포함하고 있습니다. 검색 단계에서 이 쿼리가 임베딩될 때, 쿼리의 일부 측면이 희석될 수 있으며, 따라서 반환되는 관련 청크가 이 복잡한 쿼리의 모든 구성 요소를 다루지 못할 수 있습니다.

지식 베이스를 쿼리하고 응답을 생성하기 위해 retrieve_and_generate API 호출을 사용할 것입니다. 쿼리 재구성 기능을 사용하기 위해, 아래와 같이 지식 베이스 구성에 추가 정보를 포함할 것입니다:

```
'orchestrationConfiguration': {
        'queryTransformationConfiguration': {
            'type': 'QUERY_DECOMPOSITION'
        }
    }
```
    
__참고__: 출력 응답 구조는 쿼리 재구성이 없는 일반 __retrieve_and_generate__와 동일합니다.

#### Without Query Reformulation

쿼리 재구성을 사용하지 않고 다음 쿼리에 대한 생성된 결과가 어떻게 나타나는지 살펴보겠습니다:

"Where is the Octank company waterfront building located and how does the whistleblower scandal hurt the company and its image?"

"Octank 회사의 워터프론트 빌딩은 어디에 위치해 있으며, 내부고발자 스캔들이 회사와 그 이미지에 어떤 해를 끼쳤습니까?"

In [38]:
query = "2024년 시도별 인구・가구・주택 특성을 종합적으로 비교 분석하시오.(단, 각 시도의 연령구조, 1인가구 비율, 주택 노후도, 외국인 비율을 모두 포함하여 상위 3개 지역과 하위 3개 지역을 각각 제시하고, 이들 간의 특성 차이를 설명하시오)"

In [39]:
response_ret = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)


# generated text output

print(response_ret['output']['text'],end='\n'*2)

2024년 시도별 인구・가구・주택 특성을 종합적으로 비교해 보면, 상위 3개 지역은 서울, 경기, 인천으로 나타났습니다. 이들 지역은 다음과 같은 특성을 보입니다:

- 연령구조: 경제활동인구 비중이 높고 고령인구 비중이 낮음
- 1인가구 비율: 1인가구 비율이 전국 평균보다 높음
- 주택 노후도: 신규 주택 비중이 높아 노후 주택 비율이 낮음
- 외국인 비율: 외국인 거주 비율이 높은 편임 반면 하위 3개 지역은 전남, 경북, 강원으로 나타났으며, 다음과 같은 특성이 있습니다:

- 연령구조: 고령인구 비중이 높고 경제활동인구 비중이 낮음  
- 1인가구 비율: 1인가구 비율이 전국 평균보다 낮음
- 주택 노후도: 노후 주택 비율이 높은 편임
- 외국인 비율: 외국인 거주 비율이 낮은 편임

이처럼 상위 지역과 하위 지역 간에는 인구구조, 가구 형태, 주택 환경, 외국인 비율 등 여러 측면에서 큰 차이를 보이고 있습니다.



In [40]:
response_without_qr = response_ret['citations'][0]['retrievedReferences']
print("# of citations or chunks used to generate the response: ", len(response_without_qr))
def citations_rag_print(response_ret):
#structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(response_ret,1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

citations_rag_print(response_without_qr)

# of citations or chunks used to generate the response:  2
Chunk 1:  2024년 인구주택총조사 결과      Ⅰ. 인구 ····················································································· 7      1. 총인구 ························································································ 7      가. 인구 규모 ············································································ 7      나. 성․연령별 인구 ································································· 9      다. 지역별 인구 ······································································ 12      라.

Chunk 1 Location:  {'s3Location': {'uri': 's3://standard-kb-9085801/2024_population-checkpoint.pdf'}, 'type': 'S3'}

Chunk 1 Metadata:  {'x-amz-bedrock-kb-source-uri': 's3://standard-kb-9085801/2024_population-checkpoint.pdf', 'x-amz-bedrock-kb-document-page-number': 5.0, 'x-amz-bedrock-kb-chunk-id': '1%3A0%3AELNqVZgBBdp4KKBjqZQw', 'x-amz-bedrock-kb-data-source-id': 'O7XU7FC8NC'}

Chunk 2:  117 -0 -0.3 222 경남 남해군 20 -0 -0.8     223 

위의 인용에서 볼 수 있듯이, 복잡한 쿼리를 사용한 우리의 검색은 건물과 관련된 청크는 전혀 반환하지 않고, 대신 내부고발자 사건과 가장 유사한 임베딩에만 초점을 맞추었습니다.

이는 쿼리의 임베딩 과정에서 쿼리의 해당 부분의 의미가 일부 희석되었을 수 있다는 것을 나타냅니다.

#### With Query Reformulation

이제 쿼리 재구성이 어떻게 더 정렬된 컨텍스트 검색에 도움이 될 수 있는지, 그리고 이것이 결과적으로 응답 생성의 정확성을 어떻게 향상시킬 수 있는지 살펴보겠습니다.

In [41]:
response_ret = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            },
            'orchestrationConfiguration': {
                'queryTransformationConfiguration': {
                    'type': 'QUERY_DECOMPOSITION'
                }
            }
        }
    }
)


# generated text output

print(response_ret['output']['text'],end='\n'*2)

2024년 시도별 인구, 가구, 주택 특성을 종합적으로 비교하면 다음과 같습니다:

- 인구 측면에서 서울, 경기, 인천 등 수도권 지역의 고령화율이 상대적으로 낮고, 생산연령인구 비중이 높습니다. 반면 전남, 경북 등 비수도권 지역은 고령화율이 높고 생산연령인구 비중이 낮습니다. - 가구 측면에서 1인 가구 비율은 서울, 대전 등 대도시 지역이 높고, 광주, 부산 등 광역시 지역도 1인 가구 비율이 높은 편입니다. 반면 인천, 경기 등 수도권 지역은 1인 가구 비율이 상대적으로 낮습니다. - 주택 측면에서 전남, 경북 등 비수도권 지역은 노후 주택(30년 이상) 비율이 높고, 세종, 인천 등 수도권 지역은 노후 주택 비율이 낮습니다.



쿼리 재구성을 사용하여 검색된 청크들을 살펴보겠습니다

In [42]:
response_with_qr = response_ret['citations'][0]['retrievedReferences']
print("# of citations or chunks used to generate the response: ", len(response_with_qr))
def citations_rag_print(response_ret):
#structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(response_ret,1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

citations_rag_print(response_with_qr)
print("# of citations or chunks used to generate the response: ", len(response_with_qr))

# of citations or chunks used to generate the response:  1
Chunk 1:  51.4 0.5 16.3 15.7 -0.5 39.5 41.2 1.7 242.8 261.7 18.9     경 북 50.0 50.7 0.7 15.2 14.6 -0.5 35.9 37.9 2.0 236.8 259.2 22.4     경 남 47.8 48.5 0.7 16.6 16.0 -0.6 29.2 31.0 1.8 175.7 194.0 18.3     제 주 45.1 45.9 0.7 18.6 18.0 -0.7 25.1 26.5 1.4 134.9 147.7 12.8     - 20 -< 표 9 > 시도별 인구의 연령 분포, 2023~2024년     (단위 : 천 명, %, %p)     시 도 0~14세(유소년) 15~64세(생산연령) 65세 이상(고령)     2023년 2024년

Chunk 1 Location:  {'s3Location': {'uri': 's3://standard-kb-9085801/2024_population-checkpoint.pdf'}, 'type': 'S3'}

Chunk 1 Metadata:  {'x-amz-bedrock-kb-source-uri': 's3://standard-kb-9085801/2024_population-checkpoint.pdf', 'x-amz-bedrock-kb-document-page-number': 29.0, 'x-amz-bedrock-kb-chunk-id': '1%3A0%3AYrNqVZgBBdp4KKBjqZQw', 'x-amz-bedrock-kb-data-source-id': 'O7XU7FC8NC'}

# of citations or chunks used to generate the response:  1


쿼리 재구성을 활성화했을 때, 검색된 청크들이 이제 내부고발자 스캔들과 워터프론트 건물의 위치 구성 요소 모두에 대한 컨텍스트를 제공하는 것을 볼 수 있습니다.

### CloudWatch Logs를 사용하여 프롬프트 분해 관찰하기
복잡한 쿼리는 검색을 수행하기 전에 여러 개의 하위 쿼리로 분해됩니다. 위 예제 쿼리에서 분해 작업을 위한 호출을 분리했을 때 이를 확인할 수 있으며, 여기서 __standalone_question__은 원래 쿼리이고 결과로 나온 하위 쿼리들은 __\<query\>__ 태그 사이에 표시됩니다.

__참고__: CloudWatch에서 로그를 볼 수 있으려면 Bedrock에서 호출 로깅을 활성화해야 합니다. 자세한 내용은 [여기](https://docs.aws.amazon.com/bedrock/latest/userguide/model-invocation-logging.html)를 참조하세요.

```
<generated_queries>

<standalone_question>
What is octank tower and how does the whistleblower scandal hurt the company and its image?
</standalone_question>

<query>
What is octank tower?
</query>

<query>
What is the whistleblower scandal involving Octank company?
</query>

<query>
How did the whistleblower scandal affect Octank company's reputation and public image?
</query>

</generated_queries>
```

```
<generated_queries>

<standalone_question>
옥탱크 타워는 무엇이고 내부고발자 스캔들이 회사와 그 이미지에 어떤 해를 끼쳤습니까?
</standalone_question>

<query>
옥탱크 타워는 무엇입니까?
</query>

<query>
옥탱크 회사와 관련된 내부고발자 스캔들은 무엇입니까?
</query>

<query>
내부고발자 스캔들이 옥탱크 회사의 평판과 공공 이미지에 어떤 영향을 미쳤습니까?
</query>

</generated_queries>
```

<div class="alert alert-block alert-warning">
<b>참고:</b> 불필요한 요금이 발생하지 않도록 KB(Knowledge Base), OSS 인덱스 및 관련 IAM 역할과 정책을 삭제하는 것을 잊지 마세요.
</div>

In [ ]:
print("===============================Knowledge base with fixed chunking==============================\n")
knowledge_base_standard.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)

이제 쿼리 재구성이 어떻게 작동하는지, 그리고 복잡한 쿼리에 대한 응답을 어떻게 개선할 수 있는지 살펴보았으니, 이 기술을 더 깊이 탐구하고 실험하여 RAG 워크플로우를 최적화해 보시기 바랍니다.